In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import linregress
from yelp.client import Client
import json
from ratelimit import limits
import requests
import csv
import random
from keys import my_api_key

In [4]:
df = pd.read_csv("../sub-est2019_all.csv", engine='python')
rural_city = df[(df["POPESTIMATE2019"]< 50000) & (df["POPESTIMATE2019"] > 1000) & (df["PRIMGEO_FLAG"]==0)]
rural_df = rural_city[["NAME", "STNAME", "POPESTIMATE2019"]]
urban_city = df[df["POPESTIMATE2019"]>= 50000]
urban_df = urban_city[["NAME", "STNAME", "POPESTIMATE2019"]]
urban_index = urban_df.loc[urban_df["NAME"] == urban_df["STNAME"]].index
urban_clean = urban_df.drop(urban_index)
random_urban_city = urban_clean[["NAME", "STNAME","POPESTIMATE2019"]].sample(n=100, random_state=1)
random_rural_city = rural_df[["NAME", "STNAME", "POPESTIMATE2019"]].sample(n=100, random_state=1)

In [5]:
headers = {'Authorization': 'Bearer %s' % my_api_key}

rural_dict = {
    "Searched": [],
    "City": [],
    "State": [],
    "Lat": [],
    "Lng": [],
    "Restaurant": [],
    "Category": [],
#     "Category 2": [],
#     "Category 3": [],
    "Transactions":[]
}

url='https://api.yelp.com/v3/businesses/search'
offset = np.arange(0,950,50).tolist() 

for city, state in zip(random_rural_city["NAME"], random_rural_city["STNAME"]):
    for i in range(len(offset)):
        params = {
            'location': f"{city}, {state}",
            'radius': 8047,
            'limit': 50,
            'offset': offset[i]
                 }
        response = requests.get(url, params = params, headers = headers).json()
        try:
            for business in range(len(response["businesses"])):
                try:
                    rural_dict["Searched"].append(f"{city}, {state}")
                    rural_dict["City"].append(response["businesses"][business]["location"]["city"])
                    rural_dict["State"].append(response["businesses"][business]["location"]["state"])
                    rural_dict["Lat"].append(response["businesses"][business]["coordinates"]["latitude"])
                    rural_dict["Lng"].append(response["businesses"][business]["coordinates"]["longitude"])
                    rural_dict["Restaurant"].append(response["businesses"][business]["name"])
                    rural_dict["Category"].append([response["businesses"][business]["categories"][i]["title"] for i in range(len(response["businesses"][business]["categories"]))])
#                     rural_dict["Category"].append(response["businesses"][business]["categories"][0]["title"])
#                     rural_dict["Category 2"].append(response["businesses"][business]["categories"][1]["title"])
#                     rural_dict["Category 3"].append(response["businesses"][business]["categories"][2]["title"])
                    rural_dict["Transactions"].append(response["businesses"][business]["transactions"])
                except KeyError:
                    rural_dict["Searched"].append("No City Found")
                    rural_dict["City"].append("No City Found")
                    rural_dict["State"].append("No State Found")
                    rural_dict["Lat"].append("No Lat Found")
                    rural_dict["Lng"].append("No Lng Found")
                    rural_dict["Restaurant"].append("No Restaurant Found")
                    rural_dict["Category"].append("No Categories")
                    rural_dict["Transactions"].append("No Transactions Found")
#                 except IndexError:
#                     rural_dict["Category 2"].append("")
#                     rural_dict["Category 3"].append("")
        except KeyError:
                pass

In [6]:
rural = pd.DataFrame.from_dict(rural_dict, orient='index')
rural = rural.transpose()

In [7]:
headers = {'Authorization': 'Bearer %s' % my_api_key}

urban_dict = {
    "Searched": [],
    "City": [],
    "State": [],
    "Lat": [],
    "Lng": [],
    "Restaurant": [],
    "Category": [],
#     "Category 2": [],
#     "Category 3": [],
    "Transactions":[]
}

url='https://api.yelp.com/v3/businesses/search'
offset = np.arange(0,950,50).tolist() 

for city, state in zip(random_urban_city["NAME"], random_urban_city["STNAME"]):
    for i in range(len(offset)):
        params = {
            'location': f"{city}, {state}",
            'radius': 8047,
            'limit': 50,
            'offset': offset[i]
                 }
        response = requests.get(url, params = params, headers = headers).json()
        try:
            for business in range(len(response["businesses"])):
                try:
                    urban_dict["Searched"].append(f"{city}, {state}")
                    urban_dict["City"].append(response["businesses"][business]["location"]["city"])
                    urban_dict["State"].append(response["businesses"][business]["location"]["state"])
                    urban_dict["Lat"].append(response["businesses"][business]["coordinates"]["latitude"])
                    urban_dict["Lng"].append(response["businesses"][business]["coordinates"]["longitude"])
                    urban_dict["Restaurant"].append(response["businesses"][business]["name"])
                    urban_dict["Category"].append([response["businesses"][business]["categories"][i]["title"] for i in range(len(response["businesses"][business]["categories"]))])
#                     rural_dict["Category"].append(response["businesses"][business]["categories"][0]["title"])
#                     rural_dict["Category 2"].append(response["businesses"][business]["categories"][1]["title"])
#                     rural_dict["Category 3"].append(response["businesses"][business]["categories"][2]["title"])
                    urban_dict["Transactions"].append(response["businesses"][business]["transactions"])
                except KeyError:
                    urban_dict["Searched"].append("No City Found")
                    urban_dict["City"].append("No City Found")
                    urban_dict["State"].append("No State Found")
                    urban_dict["Lat"].append("No Lat Found")
                    urban_dict["Lng"].append("No Lng Found")
                    urban_dict["Restaurant"].append("No Restaurant Found")
                    urban_dict["Category"].append("No Categories")
                    urban_dict["Transactions"].append("No Transactions Found")
#                 except IndexError:
#                     rural_dict["Category 2"].append("")
#                     rural_dict["Category 3"].append("")
        except KeyError:
                pass

In [8]:
urban = pd.DataFrame.from_dict(urban_dict, orient='index')
urban = urban.transpose()

In [9]:
urban.to_csv("../urban.csv")
rural.to_csv("../rural.csv")

In [10]:
rural_cat= pd.DataFrame(rural["Category"].to_list(), columns = ["Category 1", "Category 2", "Category 3", "Category 4", "Category 5", "Category 6"], index = rural.index)
rural_cats = rural_cat[["Category 1", "Category 2", "Category 3"]]
rural_clean = rural.merge(rural_cats, left_index = True,right_index = True)

In [11]:
urban_cat= pd.DataFrame(urban["Category"].to_list(), columns = ["Category 1", "Category 2", "Category 3", "Category 4"], index = urban.index)
urban_cats = urban_cat[["Category 1", "Category 2", "Category 3"]]
urban_clean = urban.merge(urban_cats, left_index = True,right_index = True)

In [130]:
keywords = ["Vegan", "vegan"]
rural_vegan = rural_clean[(rural_clean["Category 1"].isin(keywords))|(rural_clean["Category 2"].isin(keywords))|(rural_clean["Category 3"].isin(keywords))]
urban_vegan = urban_clean[(urban_clean["Category 1"].isin(keywords))|(urban_clean["Category 2"].isin(keywords))|(urban_clean["Category 3"].isin(keywords))]

In [131]:
rural_clean["Vegan Options"] = np.where(rural_clean["Category 1"].isin(keywords)|rural_clean["Category 2"].isin(keywords)|rural_clean["Category 3"].isin(keywords), 1, 0)
rural_clean.to_csv("../Data Pull/rural_clean.csv")

In [132]:
urban_clean["Vegan Options"] = np.where(urban_clean["Category 1"].isin(keywords)|urban_clean["Category 2"].isin(keywords)|urban_clean["Category 3"].isin(keywords), 1, 0)
urban_clean.to_csv("../Data Pull/urban_clean.csv")

In [133]:
urban_clean.to_csv("../Data Pull/urban_clean.csv")
rural_clean.to_csv("../Data Pull/rural_clean.csv")

In [134]:
df["Searched"] = df["NAME"] + ", " + df["STNAME"]

In [135]:
rural_clean_pop = pd.merge(rural_clean, df[["POPESTIMATE2019", "Searched"]],  how='left', on= "Searched")
urban_clean_pop = pd.merge(urban_clean, df[["POPESTIMATE2019", "Searched"]],  how='left', on= "Searched")

In [141]:
rural_group = rural_clean_pop.groupby("Searched")

rural_rest_count = rural_group["Restaurant"].count()
rural_vegan_count = rural_group["Vegan Options"].sum()
rural_pop = rural_group["POPESTIMATE2019"].max()
vegan_pct = rural_vegan_count/rural_rest_count * 100
vegan_percap = rural_vegan_count/rural_pop

rural_summary = pd.DataFrame({
    "Population" : rural_pop,
    "Total Restaurants" : rural_rest_count,
    "Restaurants w/ Vegan Options" : rural_vegan_count,
    "% Vegan Options" : vegan_pct,
    "Vegan Options per captia" : vegan_percap
})

rural_summary["% Vegan Options"] = rural_summary["% Vegan Options"].map("{:.0f}%".format)
rural_summary.to_csv("../Data Pull/rural_summary.csv")

In [142]:
urban_group = urban_clean_pop.groupby("Searched")

urban_rest_count = urban_group["Restaurant"].count()
urban_vegan_count = urban_group["Vegan Options"].sum()
urban_pop = urban_group["POPESTIMATE2019"].max()
vegan_pct = urban_vegan_count/urban_rest_count * 100
vegan_percap = urban_vegan_count/urban_pop

urban_summary = pd.DataFrame({
    "Population" : urban_pop,
    "Total Restaurants" : urban_rest_count,
    "Restaurants w/ Vegan Options" : urban_vegan_count,
    "% Vegan Options" : vegan_pct,
    "Vegan Options per captia" : vegan_percap
})

urban_summary["% Vegan Options"] = urban_summary["% Vegan Options"].map("{:.0f}%".format)

urban_summary.to_csv("../Data Pull/urban_summary.csv")

In [ ]:
urban_tot_cities = urban_summary["Searched"].nunique()
urban_tot_pop = urban_summary["Population"].sum()
urban_tot_rest = urban_summary["Total Restaurants"].sum()
urban_tot_vegan = urban_summary["Restaurants w/Vegan Options"].sum()
urban_tot_per = urban_tot_vegan/urban_tot_rest * 100
urban_tot_percap = urban_vegan/urban_tot_pop

urban_tot_summary = pd.DataFrame({
    "Total Cities" = urban_tot_cities
})

## Urban Scatter Plot